# Robust Optimization Final Project Code

## Declare Variables and Input Information

In [3]:
using JuMP, Gurobi, CSV, DataFrames

const GRB_ENV = Gurobi.Env()

Gurobi.Env(Ptr{Nothing} @0x000000007f18c090, false, 0)

In [17]:
dir2 = "Robot Pathing/";

# DR_input = dir2 * "Detection.csv";
# E_input = dir2 * "Adjacency.csv";
# T_input = dir2 * "Time.csv";
# robust_output = dir2 * "robust_output.csv";
# robust_path = dir2*"robust_path.csv";

# #read in the data files
# DR = CSV.read(DR_input, DataFrame, header=0);
# T = CSV.read(T_input, DataFrame, header=0);
# E = CSV.read(E_input, DataFrame, header=0);
# tri = CSV.read("100_triangles.csv", DataFrame, header=0);
# D_b = CSV.read(dir2* "Detection_buffer.csv", DataFrame, header=0);
# n = size(T)[1];
w=11
h=11
wh_string=string(w)*"_"*string(h)
scenario_name = "test"
arcs_file="arcs_"*wh_string*"_"*scenario_name*".csv";
tri_file=wh_string*"_triangle.csv"
# CS_file = wh_string*"_CS.csv"
arcs= CSV.read(arcs_file, DataFrame, header=0);
tri= CSV.read(tri_file, DataFrame, header=0);
# CS = CSV.read(CS_file, DataFrame, header=0);
ins = CSV.read(wh_string*"_inflow.csv", DataFrame, header=0);
outs = CSV.read(wh_string*"_outflow.csv", DataFrame, header=0);
batteryCapacity = 200*(2*w*h)

48400

In [18]:
D_lin = arcs[!,4];
T_lin = arcs[!,5];
E_lin = arcs[!,6];
# C=CS[!,1];

tris=[]
for t in eachrow(tri)
    new_t=[]
    for a in t
        if a!=0
            append!(new_t,a)
        end
    end
    push!(tris,new_t)
end
IN=size(ins,1)
inflow=[]
outflow=[]
for i in 1:IN
    new_in=[]
    new_out=[]
    for j in 1:10
        if ins[i,j]!=0
            append!(new_in,ins[i,j])
            append!(new_out,outs[i,j])
        end
    end
    push!(inflow,new_in)
    push!(outflow,new_out)
end        

In [89]:
# D_lin, T_lin, e_code, tris = linearizeModel(100,100,E,DR,T,tri);

In [19]:
#User Input and Initial Calculations
τ = 60*60*60; #number of seconds to traverse the graph
# S_max=7
# C_max=5
s = 1 #the starting node of the troops
f = 121 #the destination node of the troops
N=w*h;
A=28*w*h-18*(w+h)+12;
A=4-6(w+h)+ 8*(w*h)

840

## Solve the Model

In [21]:
@time begin
    pathrob,objT,sTime,objD=two_step_en(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow)
    end 

write_path(pathrob, A)

Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
  0.217944 seconds (107.93 k allocations: 7.270 MiB)


"output_test.csv"

In [27]:
@time begin
    pathrob,objT,sTime,objD=two_step(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow)
    end 

Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
  5.953168 seconds (2.22 M allocations: 118.039 MiB, 2.25% gc time, 96.43% compilation time)


([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 59.799961640843605, time, 2.1986803057871187)

In [12]:
write_path(pathrob, A)

"output_3 Enemies.csv"

## Common Functions

In [15]:
function two_step(w,h,s,f,A,N,d,t,tris,τ,inflow,outflow)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=600
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-4, "OutputFlag" => 0);
    @variable(m, x[1:A], Bin)
    @objective(m, Min, sum(d[i]*x[i] for i in 1:A))
    
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in outflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i] if i!=f && i!=s)-sum(x[k] for k in outflow[i] if i!=f && i!=s)==0)
    @constraint(m, sum(x[k] for k in inflow[s])-sum(x[k] for k in outflow[s])==-1)
    @constraint(m, sum(x[k] for k in inflow[f])-sum(x[k] for k in outflow[f])==1)

    tri_tot=length(tris)
    @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    @constraint(m, sum(t[i]*x[i] for i=1:A) <= τ)
    optimize!(m)
    #time=MOI.get(m, MOI.SolveTime()) 
    D_opt=objective_value(m)
    
    @objective(m, Min, sum(t[i]*x[i] for i in 1:A))
    @constraint(m, sum(d[i]*x[i] for i in 1:A)<=D_opt)
    optimize!(m)
    optimalpath = value.(x)
    objective = objective_value(m)
    #time+=MOI.get(m, MOI.SolveTime())
    return optimalpath, objective, time, D_opt
end

function two_step_en(w,h,s,f,A,N,d,t,E,tris,τ,inflow,outflow)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=600
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-4, "OutputFlag" => 0);
    @variable(m, x[1:A], Bin)
    @objective(m, Min, sum(d[i]*x[i] for i in 1:A))
    
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in outflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i] if i!=f && i!=s)-sum(x[k] for k in outflow[i] if i!=f && i!=s)==0)
    @constraint(m, sum(x[k] for k in inflow[s])-sum(x[k] for k in outflow[s])==-1)
    @constraint(m, sum(x[k] for k in inflow[f])-sum(x[k] for k in outflow[f])==1)

    tri_tot=length(tris)
    @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    @constraint(m, sum(t[i]*x[i] for i=1:A) <= τ)
    @constraint(m, sum(E[i]*x[i] for i=1:A) <= batteryCapacity)
    optimize!(m)
    #time=MOI.get(m, MOI.SolveTime())
    D_opt=objective_value(m)
    
    @objective(m, Min, sum(E[i]*x[i] for i in 1:A))
    @constraint(m, sum(d[i]*x[i] for i in 1:A)<=D_opt)
    optimize!(m)
    optimalpath = value.(x)
    objective = objective_value(m)
    #time+=MOI.get(m, MOI.SolveTime())
    return optimalpath, objective, time, D_opt
end

function one_step(w,h,s,f,A,N,S_m,C_m,d,t,tris,τ,C,S,inflow,outflow)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=600
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-4, "OutputFlag" => 0);
    @variable(m, x[1:A], Bin)
    @objective(m, Min, sum(d[i]*x[i] for i in 1:A))
    
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in outflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i] if i!=f && i!=s)-sum(x[k] for k in outflow[i] if i!=f && i!=s)==0)
    @constraint(m, sum(x[k] for k in inflow[s])-sum(x[k] for k in outflow[s])==-1)
    @constraint(m, sum(x[k] for k in inflow[f])-sum(x[k] for k in outflow[f])==1)

    tri_tot=length(tris)
    @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    C_tot=length(C)
    @constraint(m, sum(x[k] for k in C)<=C_max)
    @constraint(m, sum(x[k] for k in S)<=S_max)
    @constraint(m, sum(t[i]*x[i] for i=1:A) <= τ)
    optimize!(m)
    time=MOI.get(m, MOI.SolveTime())
    optimalpath = value.(x)
    objective = objective_value(m)
    objT=sum(optimalpath.*t)
    return optimalpath, objective, time, objT
end
function write_path(opt_path, A)
    path=[]
    for i in 1:A
        if opt_path[i]==1
            append!(path,i)
        end
    end
    K=length(path)
    path_table=zeros(K,2)
    for i in 1:K
        path_table[i]=path[i]
    end
    path_output3 = DataFrame(path_table, :auto);
    CSV.write("output_"*scenario_name*".csv", path_output3); 
end

write_path (generic function with 1 method)

In [45]:
function write_opt_path(opt_path, N)
    pathrob_key=zeros(N,2)
    node_path=[]
    for i in 1:N
        pathrob_key[i,1]+=i
        pathrob_key[i,2]=pathrob[i]
        if pathrob[i]>0
            append!(node_path,findall(x->x==i,e_code))
        end
    end
    K=size(node_path)[1]
    node_path_table=zeros(K,2)
    for k in 1:K
        node_path_table[k,1]+=node_path[k][1]
        node_path_table[k,2]+=node_path[k][2]
    end

    path_output3 = DataFrame(node_path_table);
    CSV.write(robust_path, path_output3); 
end

function linearizeModel(w,h,E,D,T,Tri)
    N=28*w*h-18*(w+h)+12
    D_lin=zeros(N)
    T_lin=zeros(N)
    k=1
    e_code=Matrix(E)
    for i in 1:n
        for j in 1:n
            if E[i,j]==1
                e_code[i,j]=k
                D_lin[k]+=D[i,j]
                T_lin[k]+=T[i,j]
                k+=1
            end   
        end
    end
    tris=[]
    rows,cols=size(Tri)
    for row in 1:rows
        newtri=[]
        for col in 1:cols
            ijstring=Tri[row,col]
            comma=findnext(isequal(','),ijstring,1)
            i=parse(Int64,ijstring[2:(comma-1)])
            j=parse(Int64,ijstring[(comma+1):(length(ijstring)-1)])
            if i!=0 && j!=0
                append!(newtri,e_code[i,j])
            end
        end
        push!(tris,newtri)
    end
    return D_lin, T_lin, e_code, tris
end

function solve_two_step(w,h,s,f,S_m,C_m,d,T,code,tris,use_tris)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=600
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-4, "OutputFlag" => 0);
    N=28*w*h-18*(w+h)+12; 
    @variable(m, x[1:N], Bin)
    @objective(m, Min, sum(d[i]*x[i] for i in 1:N))
    e_codeT=e_code'
    crawl=[]
    sneak=[]
    for i in 1:n
        inflow=[]
        outflow=[]
        for j in 1:n
            if e_code[i,j]>0
                append!(inflow,e_code[i,j])
            end
            if e_codeT[i,j]>0
                append!(outflow,e_codeT[i,j])
            end
            if j<=w*h && i<=w*h && e_code[i,j]>0
                append!(crawl,e_code[i,j])
            elseif j>=(w*h+1) && i>=(w*h+1) && j<=2*w*h && i<=2*w*h && e_code[i,j]>0
                append!(sneak,e_code[i,j])
            end
        end
        @constraint(m, sum(x[k] for k in inflow) <= 1)
        @constraint(m, sum(x[k] for k in outflow) <= 1)
        if i==s
            @constraint(m, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==-1)
        elseif i==f
            @constraint(m, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==1)
        else
            @constraint(m, sum(x[k] for k in inflow)==sum(x[k] for k in outflow))
        end
    end
    if use_tris
        tri_tot=length(tris)
        @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    end

    @constraint(m, sum(x[k] for k in crawl)<=C_max)
    @constraint(m, sum(x[k] for k in sneak)<=S_max)
    @constraint(m, sum(T[i]*x[i] for i=1:N) <= τ)
    optimize!(m)
    time=MOI.get(m, MOI.SolveTime())
    D_opt=objective_value(m)
    
    @objective(m, Min, sum(T[i]*x[i] for i in 1:N))
    @constraint(m, sum(d[i]*x[i] for i in 1:N)<=D_opt)
    optimize!(m)
    optimalpath = value.(x)
    objective = objective_value(m)
    time+=MOI.get(m, MOI.SolveTime())
    return optimalpath, objective, time, D_opt
end                                           

solve_two_step (generic function with 1 method)

## Other Functions during Development

In [46]:
function solve_accelerated(w,h,s,f,S_m,C_m,D,T,code,tris,α)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=180
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-4, "OutputFlag" => 0);
    N=28*w*h-18*(w+h)+12; 
    @variable(m, x[1:N]>=0)
    @objective(m, Min, sum((D[i]+α*T[i])*x[i] for i in 1:N))
    b=[]
    e_codeT=e_code'
    crawl=[]
    sneak=[]
    c=Vector{ConstraintRef}(undef, 2)
    for i in 1:n
        inflow=[]
        outflow=[]
        for j in 1:n
            if e_code[i,j]>0
                append!(inflow,e_code[i,j])
            end
            if e_codeT[i,j]>0
                append!(outflow,e_codeT[i,j])
            end
            if j<=w*h && i<=w*h && e_code[i,j]>0
                append!(crawl,e_code[i,j])
            elseif j>=(w*h+1) && i>=(w*h+1) && j<=2*w*h && i<=2*w*h && e_code[i,j]>0
                append!(sneak,e_code[i,j])
            end
        end
        if i==s
            c[1] = @constraint(m, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==-1)
        elseif i==f
            c[2] = @constraint(m, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==1)
        else
            @constraint(m, sum(x[k] for k in inflow)==sum(x[k] for k in outflow))
        end
    end
    tri_tot=length(tris)
    @constraint(m, c_tri[t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)

    @constraint(m, c_crawl, sum(x[k] for k in crawl)<=C_max)
    @constraint(m, c_sneak, sum(x[k] for k in sneak)<=S_max)
    @constraint(m, c_time, sum(T[i]*x[i] for i=1:N) <= τ)
    @constraint(m, lp[i in 1:N], x[i]<=1)

    MP_time = []; SP_max_time = []; SP_time = []
    cutOff=[]
    
    while true
        push!(MP_time, @elapsed optimize!(m))
        println(MP_time)
        if termination_status(m) == MOI.OPTIMAL
            optimalpath = value.(x)
            objective = objective_value(m)
            time=MOI.get(m, MOI.SolveTime())
            return optimalpath, objective, time, cutOff
        end
        z = sum((D[i]+α*T[i])*x[i] for i in 1:N)
        pb=(-getdual(c[1])+getdual(c[2])+C_max*getdual(c_crawl)+S_max*getdual(c_sneak)
            +τ*getdual(c_time)+sum(getdual(lp[i]) for i in 1:N)+sum(getdual(c_tri[t]) for t in 1:tri_tot))
        z_pb=z-pb
        println(z_pb)
        for i in 1:N
            if D[i]>z_pb
                @constraint(m,x[i]==0)
                append!(cutOff,i)
            end
        end
        MAXTIME+=1
        set_optimizer_attributes(m, "TimeLimit" => MAXTIME);
    end
    
end

function solveRobust_Linear_added(N::Int, n::Int, s::Int, f::Int, D, T, e_code, Dlim,α,tris,w,h)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[i=1:N], Bin)
    
    @objective(model, Min, sum(D[i]*x[i] for i=1:N)+α*sum(T[i]*x[i] for i=1:N))
    e_codeT=e_code'
    crawl=[]
    sneak=[]
    for i in 1:n
        inflow=[]
        outflow=[]
        for j in 1:n
            if e_code[i,j]>0
                append!(inflow,e_code[i,j])
            end
            if e_codeT[i,j]>0
                append!(outflow,e_codeT[i,j])
            end
            if j<=w*h && i<=w*h && e_code[i,j]>0
                append!(crawl,e_code[i,j])
            elseif j>=(w*h+1) && i>=(w*h+1) && j<=2*w*h && i<=2*w*h && e_code[i,j]>0
                append!(sneak,e_code[i,j])
            end
        end
        if i==s
            @constraint(model, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==-1)
        elseif i==f
            @constraint(model, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==1)
        else
            @constraint(model, sum(x[k] for k in inflow)==sum(x[k] for k in outflow))
        end
    end
    tri_tot=length(tris)
    for t in 1:tri_tot
        @constraint(model, sum(x[t] for t in tris[t])<=1)
    end
    @constraint(model, sum(x[k] for k in crawl)<=C_max)
    @constraint(model, sum(x[k] for k in sneak)<=S_max)
    
     
    @constraint(model, sum(T[i]*x[i] for i=1:N) <= τ)
    
    if Dlim<1
        for i in 1:N
            if D[i]>Dlim
                @constraint(model, x[i]==0)
            end
        end
    end

    optimize!(model)
    optimalpath = value.(x)
    objective = objective_value(model)
    time=MOI.get(model, MOI.SolveTime())
    return optimalpath, objective, time
end

function solveRobust(n::Int, s::Int, f::Int, D, T, E)
    @time begin
        model = Model(Gurobi.Optimizer)

        @variable(model, x[i=1:n, j=1:n], Bin)
        @variable(model, z >= 0)

        @objective(model, Min, z)
        @constraint(model, sum((D[i,j]) * x[i,j] for i=1:n, j=1:n) <= z)

        @constraint(model, sum(x[i,j] for i=1:900,j=1:900) <= C_max)
        @constraint(model, sum(x[i,j] for i=901:1800,j=901:1800) <= S_max)

        @constraint(model, sum(T[i,j]*x[i,j] for i=1:n,j=1:n) <= τ)
        @constraint(model, [i=1:n,j=1:n], x[i,j] <= E[i,j])

        @constraint(model, sum(x[s,j] for j=1:n) == 1)
        @constraint(model, sum(x[i,f] for i=1:n) == 1)
        @constraint(model, [j=1:n; j!=s && j!=f], sum(x[i,j] for i=1:n if i!=f) == sum(x[j,k] for k=1:n if k!=s))

        optimize!(model)
        optimalpath = value.(x)
        objective = objective_value(model)
        time=MOI.get(model, MOI.SolveTime())
    end
    return optimalpath, objective, time
end
                            
function solveRobust2(n::Int, s::Int, f::Int, D, T, E,Dlim)
    model = Model(Gurobi.Optimizer)

    @variable(model, x[i=1:n, j=1:n], Bin)
    @variable(model, z >= 0)
    
    @objective(model, Min, z)
    @constraint(model, sum((D[i,j]) * x[i,j] for i=1:n, j=1:n) <= z)
    
    @constraint(model, sum(x[i,j] for i=1:900,j=1:900) <= C_max)
    @constraint(model, sum(x[i,j] for i=901:1800,j=901:1800) <= S_max)
    
    @constraint(model, sum(T[i,j]*x[i,j] for i=1:n,j=1:n) <= τ)
    @constraint(model, [i=1:n,j=1:n], x[i,j] <= E[i,j])
    
    @constraint(model, sum(x[s,j] for j=1:n) == 1)
    @constraint(model, sum(x[i,f] for i=1:n) == 1)
    @constraint(model, [j=1:n; j!=s && j!=f], sum(x[i,j] for i=1:n if i!=f) == sum(x[j,k] for k=1:n if k!=s))
    for i=1:n
        for j=1:n
            if D[i,j]>Dlim
                @constraint(model, x[i,j]==0)
            end
        end
    end
#     @constraint(model, [j=1:n,i=1:n], x[i,j]<=1+Dlim-D[i,j])
    optimize!(model)
    optimalpath = value.(x)
    objective = objective_value(model)
    time=MOI.get(model, MOI.SolveTime())
    return optimalpath, objective, time
end

function solveRobust_Linear(N::Int, n::Int, s::Int, f::Int, D, T, e_code, Dlim,α)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[i=1:N], Bin)
    
    @objective(model, Min, sum(D[i]*x[i] for i=1:N)+α*sum(T[i]*x[i] for i=1:N))
    e_codeT=e_code'
    crawl=[]
    sneak=[]
    for i in 1:n
        inflow=[]
        outflow=[]
        for j in 1:n
            if e_code[i,j]>0
                append!(inflow,e_code[i,j])
            end
            if e_codeT[i,j]>0
                append!(outflow,e_codeT[i,j])
            end
            if j<=900 && i<=900 && e_code[i,j]>0
                append!(crawl,e_code[i,j])
            elseif j>=901 && i>=901 && j<=1800 && i<=1800 && e_code[i,j]>0
                append!(sneak,e_code[i,j])
            end
        end
        if i==s
            @constraint(model, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==-1)
        elseif i==f
            @constraint(model, sum(x[k] for k in inflow)-sum(x[k] for k in outflow)==1)
        else
            @constraint(model, sum(x[k] for k in inflow)==sum(x[k] for k in outflow))
        end
    end
    @constraint(model, sum(x[k] for k in crawl)<=C_max)
    @constraint(model, sum(x[k] for k in sneak)<=S_max)
    
     
#     @constraint(model, sum(T[i]*x[i] for i=1:N) <= τ)
    
    for i in 1:N
        if D[i]>Dlim
            @constraint(model, x[i]==0)
        end
    end

    optimize!(model)
    optimalpath = value.(x)
    objective = objective_value(model)
    time=MOI.get(model, MOI.SolveTime())
    return optimalpath, objective, time
end

solveRobust_Linear (generic function with 1 method)

# Continuous Version

In [ ]:
w_field=convert(Matrix,CSV.read("walking_detection_field.csv", DataFrame, header=0));
s_field=convert(Matrix,CSV.read("sneaking_detection_field.csv", DataFrame, header=0));
c_field=convert(Matrix, CSV.read("crawling_detection_field.csv", DataFrame, header=0));
det_fields=[w_field,s_field,c_field];

In [ ]:
S=[]
r=[15,3.04,23.9]
for i in 1:3
    sTemp=[0]
    k=1
    for j in 1:50
        append!(sTemp, k)
        k+=1
    end
    push!(S,sTemp)
end
S

In [ ]:
test=[1,2,3,4,1,1,2]

In [ ]:
for i in test && test[i]<=2
    println(i)
end